In [1]:
# Libraries

from time import time as now
from itertools import combinations

In [2]:
# Initial time

initial_time = now()

# Reading the data

## Checkin
In this step, two dictionaries will be created:
1. ```checkin```, format:

```
checkin = {
        user_1: {
            location_1_1: (first and last date user_1 was in location_1_1),
            ...,
            location_1_k: (first and last date user_1 was in location_1_k)
        },
        ...
        user_n: {
            location_n_1: (first and last date user_n was in location_n_1),
            ...,
            location_n_k: (first and last date user_n was in location_n_k)
        }
}

```

2. ```location_dict```,  format:

```
location_dict = {
        location_1: {set of users who were at location_1},
        ...,
        location_n: {set of users who were at location_n}
}

```

In [3]:
# Initialize the dictionaries
checkin = dict()
location_dict_raw = dict()

# Set to store all the users who have checked-in
travelers = set()

# Read the file
with open('Gowalla_totalCheckins.txt', 'r') as f:
    for line in f:
        line = line.split('\t')
        user = int(line[0])
        travelers.add(user)
        
        # It is not neccesary to parse the date because of the format
        checkin_time = line[1]
         
        # The location_id doesn't work well, so the hash of the coordinates is used
        location = hash((line[2], line[3]))
        
        # Fill the location dictionary
        try:
            location_dict_raw[location] = location_dict_raw[location].union(set([user]))
        except KeyError:
            location_dict_raw[location] = set([user])
        
        # Fill the check-in dictionary
        try:
            # This step could be simpler because the check-in data is organized by
            # user_id and time, but I decided to do it more general
            checkin[user][location] = (min(checkin_time, checkin[user][location][0]),
                                       max(checkin_time, checkin[user][location][1]))
        except KeyError:
            try:
                checkin[user][location] = (checkin_time, checkin_time)
            except KeyError:
                checkin[user] = dict()
                checkin[user][location] = (checkin_time, checkin_time)

# Ignore locations only visited by one user
location_dict = {key: val for key, val in location_dict_raw.items() if len(val) != 1}

## Friends
In this step, one dictionary will be created
1. ```friends```, format:

```
friends = {
    user_1: [user for user in users where user > user_1 and (user, user_1) are friends],
    ...,
    user_n: [user for user in users where user > user_n and (user, user_n) are friends]
}

```

In [4]:
# Initialize the dictionary
friends = dict()

# Read the friends file
with open('Gowalla_edges.txt', 'r') as f:
    for line in f:
        line = line[:-1].split('\t')
        user1 = int(line[0])
        user2 = int(line[1])
        
        # Ignore all users who never checked-in
        if user1 not in travelers:
            continue
        
        # Ignore all friends who have already been listed
        if user1 > user2:
            continue
            
        # Ignore all friends who have never checked-in 
        if user2 not in travelers:
            continue
        
        # Fill the dictionary
        try:
            friends[user1] = friends[user1] + [user2]
        except KeyError:
            friends[user1] = [user2]

# Solution

In [5]:
# Funtions to use

def stalker_score(user1, user2, max_value=-1):
    """
    Calculate the stalker scores for the pairs
    (user1, user2) and (user2, user1).

    If both of the stalker scores would be less
    than max_value, the function returns (-1, -1)
    before iterating
    """
    sc1, sc2 = 0, 0

    # Avoid unnecessary iterations
    if len(locs1 := checkin[user1]) <= max_value \
            or len(locs2 := checkin[user2]) <= max_value\
            or len(locations := [location for location in locs1 if location in locs2]) <= max_value:
        return -1, -1
    else:
        for location in locations:
            # First and last time user1 was in location
            first_time1, last_time1 = checkin[user1][location]

            # First and last time user2 was in location
            first_time2, last_time2 = checkin[user2][location]

            if last_time1 > first_time2:  # If user2 was in location before user1
                sc2 += 1

            if last_time2 > first_time1:  # If user1 was in location before user2
                sc1 += 1
        return sc1, sc2


def possible_users(location_dict):
    """
    Generate all the possible pairs of users
    that have been in the same location at 
    least one time.
    """
    for it in (combinations(location_dict[loc], 2) for loc in location_dict):
        for user1, user2 in it:
            yield user1, user2


## 1. A & B are friends

In [6]:
max_val = -1

for user1 in friends:
    for user2 in friends[user1]:
        sc1, sc2 = stalker_score(user1, user2, max_val)
        if sc1 > sc2:
            if sc1 > max_val:
                max_val = sc1
                stalker = (user1, user2)
        elif sc2 > max_val:
            max_val = sc2
            stalker = (user2, user1)

### Answer 1



In [7]:
print(f'The friend pair {stalker} has the highest stalker score: {max_val}')

The friend pair (10410, 10393) has the highest stalker score: 365


## 2. A & B are not friends

In [8]:
# Initialize generator of all possible users pairs
user_pairs = possible_users(location_dict)

while True:
    try:
        user1, user2 = next(user_pairs)
    except StopIteration:
        break
    # Assuming that this score would be greather than the
    # last one, the first max_val used is the previous answer.
    # Otherwise, -1 should be assigned to max_val and in each
    # iteration would be necessary to check if user_1 and 
    # user_2 are friends.
    sc1, sc2 = stalker_score(user1, user2, max_val)
    
    if sc1 > sc2:
        if sc1 > max_val:
            max_val = sc1
            stalker = (user1, user2)
    elif sc2 > max_val:
        max_val = sc2
        stalker = (user2, user1)

### Answer 2

In [9]:
print(f'The non-friend pair {stalker} has the highest stalker score: {max_val}')

The non-friend pair (1251, 106819) has the highest stalker score: 384


In [10]:
# Execution time
execution_time = now() - initial_time

print(f'The total running time was {int(execution_time//60)} minutes and {int((execution_time)) % 60} seconds to run')

The total running time was 2 minutes and 19 seconds to run
